In [ ]:
%matplotlib inline
import json 
import matplotlib.pyplot as plt
import os
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import keras 

# import numpy
# import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf


from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

import glob

from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix


from keras import backend as K

# Main configs

In [ ]:
loadFromMerged=True
loadFromIndexes= False
Mapper='S'
IgnoreEmpty= True
FoldID ="1"
Epoch_count=100
Batch_size=5000

# Load the data the old way


In [3]:
# x_data= [] 
# y_data= [] 


# with open( '../files/txt/seq_mapping_large.txt' ) as f:
#     x_data =   f.readlines()

# with open( '../files/txt/command_mapping_large.txt' ) as f:
#     y_data = f.readlines()
    
    
# x_data =[ np.array([ int(y) for y in x.strip().split( ' ') ])   for x in  x_data ] 
# y_data =[ x.strip().split(' ') for x in  y_data ] 


#  Load The Data The New Way

In [4]:
#  mapps the input records to a integer array for the input
def mapping_x( inp, includeDirection = False , TrimAt= 15 ):
    if includeDirection:
        return np.array([ int(x["packet_length"]) * (1 if x['packet_source']=='hub' else -1)  for x in inp ][:15])
    else:
        return np.array([ int(x["packet_length"])  for x in inp ][:15])

In [5]:
def mapping_y_service(inp):
    return np.array(  list(set([x["event"] for x in inp])) if (len(inp )>0) else ["none"] )

def mapping_y_service_event(inp):
    return np.array(  list(set([ "%s-%s"%( x["event"] ,x["val"] ) for x in inp])) if (len(inp )>0) else ["none"] )

def mapping_y_device_service(inp):
    return np.array(  list(set([ "%s & %s"%( x["device"] ,x["event"] ) for x in inp])) if (len(inp )>0) else ["none"] )

def mapping_y_full(inp):
    return np.array(  list(set([ "%s & %s & %s"%( x["device"] ,x["event"], x['val'] ) for x in inp])) if (len(inp )>0) else ["none"] )


In [6]:
# this cleans the data removing emply nodes and turning the nodes into sarrays by calling the mapping function 
def clean_data( x_data, y_data , removeempty=True, Mapping='S'):
    cleans = [] 
    cleans = (sorted([ x for x in y_data if (removeempty and len(y_data[x]) > 0) or not removeempty  ] ))
    
    ret_x  = [x_data[x] for x in cleans]
    ret_y  = [y_data[x] for x in cleans] 
    
    print( len(y_data), len(cleans) )
    
    ret_x  = [ mapping_x(x) for x in ret_x ] 
    ret_y_s = [ mapping_y_service(y) for y in ret_y ]
    if Mapping=='S':
        ret_y  = [ mapping_y_service(y) for y in ret_y ]
    elif Mapping=='SE':
        ret_y  = [ mapping_y_service_event(y) for y in ret_y ]
    elif Mapping=='DS':
        ret_y  = [ mapping_y_device_service(y) for y in ret_y ]
    elif Mapping=='F':
        ret_y  = [ mapping_y_full(y) for y in ret_y ]
    return ret_x, ret_y, ret_y_s

In [48]:
x= []
y= []

x_test = []
y_test = []
y_test_service= []

x_train = {}
y_train = {}

test_names = []

add_to_trainig = [0,2]

if loadFromMerged:
    print("in load from merged")
    with open(  '../files/train/test/test_homes/final_upload/Merged_final_with_home/merged_hub_segments_final.json'  ) as f:
        y_data = json.load(f)

    with open(  '../files/train/test/test_homes/final_upload/Merged_final_with_home/merged_pcap_segments_final.json'  ) as f:
        x_data = json.load(f)
        
#     with open(  '../files/train/merged/hub_segments_2.json'  ) as f:
#         y_data = json.load(f)

#     with open(  '../files/train/merged/pcap_segments_2.json'  ) as f:
#         x_data = json.load(f)
  
    if len( y_data ) != len(x_data) :
        print( pick )
        
    
    x_train,y_train, y_train_service= clean_data( x_data, y_data, IgnoreEmpty , Mapping=Mapper )
    
    #     continue
#     if loadFromIndexes:
#         print("load from indexes")
#         with open("../files/train/merged/items_2_test-train_indexes.json")  as f:
#             index_info = json.load(f)


#         for i in index_info[FoldID]["test"]:
#             x_test[str(i)]=(x_data[str(i)] )
#             y_test[str(i)]=(y_data[str(i)] )

#         for i in index_info[FoldID]["train"]:
#             x_train[str(i)]=(  x_data[str(i)] )
#             y_train[str(i)]=(  y_data[str(i)] )
        
#         x_test_t,y_test_t= clean_data( x_test, y_test, IgnoreEmpty , Mapping=Mapper)
#         x_test.append(x_test_t)
#         y_test.append(y_test_t)
    #     else :
    print("loading from test files")
    test_files = sorted(glob.glob( '../files/train/test/test_homes/final_upload/usecases/pcap_segments_final_final/home*.json' ))
    print( "found files : " , len(test_files) )
    for pick  in test_files:
        fname  = os.path.basename(pick)
        test_names.append( fname )
        with open( os.path.join( '../files/train/test/test_homes/final_upload/usecases/hub_segments_final_final/', fname) ) as f:
            y_data_test = json.load(f)

        with open( os.path.join('../files/train/test/test_homes/final_upload/usecases/pcap_segments_final_final/', fname) ) as f:
            x_data_test = json.load(f)


        t_x,t_y, t_z= clean_data( x_data_test, y_data_test, False , Mapping=Mapper )

#         if test_files.index(pick) in add_to_trainig:
#             x_test_t,y_test_t, y_test_service_t= clean_data( x_data_test, y_data_test, IgnoreEmpty , Mapping=Mapper)
#             x_train.extend(x_test_t)
#             y_train.extend(y_test_t)
#             y_train_service.extend(y_test_service_t)


        x_test.append(t_x)
        y_test.append(t_y)
        y_test_service.append(t_z)
            
#     x_test = x_data[ index_info["1"]["test"]  ]
#     y_test = y_data[ index_info["1"]["test"]  ]
    
#     x_train = x_data[ index_info["1"]["train"]  ]
#     y_train = y_data[ index_info["1"]["train"]  ]
#     x.extend(t_x)
#     y.extend(t_y)
else:
    for pick in sorted(glob.glob( '../files/train/hub_segments/*.json' )):
        fname  = os.path.basename(pick)
        test_names.append( fname )
        with open( os.path.join( '../files/train/hub_segments/', fname) ) as f:
            y_data = json.load(f)

        with open( os.path.join('../files/train/pcap_segments/', fname) ) as f:
            x_data = json.load(f)

        if len( y_data ) != len(x_data) :
            print( pick )
            continue

        t_x,t_y= clean_data( x_data, y_data, True )

        x.extend( t_x)
        y.extend(t_y)

x= np.array(x)
y= np.array(y)

# x_train = np.append( x_train, x_test[0] , axis=0)
# x_train = np.append( x_train, x_test[2] , axis=0)

# y_train = np.append( y_train, y_test[0] , axis=0)
# y_train = np.append( y_train, y_test[2] , axis=0)


len(x_train), len(y_test)

in load from merged
58958 57867
loading from test files
found files :  4
32069 32069
19968 19968
9109 9109
6404 6404


(57867, 4)

In [49]:
import random
def purturb( x_inp , y_inp  ,percentage= 0.5, ignore_empty=False, overlap_count = 1 ):
    
    if ignore_empty :
        indexes= [ i for i in range(len(y_inp)) if  len( x_inp[i] )>overlap_count and  'none' not in  y_inp[i]  and 'unknown'  not in y_inp[i] ]
    else :
        indexes=  [ i for i in range(len(y_inp)) if  len( x_inp[i] )>overlap_count ]
    
    assert len(indexes)> overlap_count
    
    pickable_len =int( len(indexes) * percentage)
    if pickable_len %2 != 0 :
        pickable_len+=1
    
    indexes =  random.sample(indexes, pickable_len)
    
    set_1 = np.array( random.sample( indexes, int(pickable_len/2) ), dtype=int )
    indexes = np.array(indexes, dtype=int)
    set_2 = np.array(  indexes[ ~np.isin(indexes, set_1) ], dtype=int )
    
    x_new = list(x_inp) 
    y_new = list(y_inp)
    for i in range(len(set_1)):
        a= x_inp[set_1[i]]
        b= x_inp[set_2[i]]
        a_y =list( y_inp[set_1[i]])
        b_y = list(y_inp[set_2[i]])
        
        ret = [ a[:-1* overlap_count] , b[:overlap_count] ,a[-1*overlap_count:],b[overlap_count:]]
        ret  = [  i if type(i)==np.ndarray else np.array([i]) for i in ret ]      
        ret =np.array(list(np.concatenate(ret)))
#         print(a,b, ret)

        x_new.append( ret)
        y_new.append( list(set(b_y + a_y))) 

    return x_new, y_new

In [50]:
# x_train, y_train = purturb( x_train, y_train, ignore_empty=True,overlap_count=2 )
x_train, y_train = purturb( x_train, y_train, ignore_empty=True,overlap_count=1 )

In [10]:
list(np.unique(  np.concatenate( y_train  )))

['acceleration',
 'activity',
 'battery',
 'button',
 'colorTemperature',
 'contact',
 'level',
 'lock',
 'motion',
 'ping',
 'status',
 'switch',
 'temperature',
 'threeAxis',
 'unknown',
 'water']

In [11]:
classes = sorted(list(np.unique(  np.concatenate( y_train  ))))
print([ (i , classes[i]) for i in range( len(classes) ) ])

service_classes = sorted(list(np.unique(  np.concatenate( y_train_service  ))))
[ (i , service_classes[i]) for i in range( len(service_classes) ) ]

[(0, 'acceleration'), (1, 'activity'), (2, 'battery'), (3, 'button'), (4, 'colorTemperature'), (5, 'contact'), (6, 'level'), (7, 'lock'), (8, 'motion'), (9, 'ping'), (10, 'status'), (11, 'switch'), (12, 'temperature'), (13, 'threeAxis'), (14, 'unknown'), (15, 'water')]


[(0, 'acceleration'),
 (1, 'activity'),
 (2, 'battery'),
 (3, 'button'),
 (4, 'colorTemperature'),
 (5, 'contact'),
 (6, 'level'),
 (7, 'lock'),
 (8, 'motion'),
 (9, 'ping'),
 (10, 'status'),
 (11, 'switch'),
 (12, 'temperature'),
 (13, 'threeAxis'),
 (14, 'unknown'),
 (15, 'water')]

In [ ]:
[ (i , classes[i]) for i in range( len(classes) ) ]

In [ ]:
def is_clean_event( inp, return_clean= True  ):
    if return_clean:
        return  'no_logs' not in inp and 'lock-unlocked' not in inp and 'on/off-XXX' not in inp and 'raw-XXX' not in inp and 'read_attr_-_raw-XXX' not in inp
    else:
        return  'lock-locked' in inp or 'lock-unlocked'  in inp or 'on/off-XXX' in inp or  'raw-XXX' in inp  or 'read_attr_-_raw-XXX' in inp 
    
def is_clean_service( inp, return_clean= True  ):
    
    if return_clean:
        return  'no_logs' not in inp and 'unknown' not in inp and 'read_attr_-_raw' not in inp #and 'ping' not in inp 
    else:
        return  'no_logs' in inp or  'unknown' in inp  or 'read_attr_-_raw' in inp #or 'ping' in inp 
#     return  "contact-open" not in inp and 'contact-closed' not in inp

In [ ]:

y_test[0][0]

In [ ]:
toKeep = [ i for i in range(len(y_train)) if is_clean_event( y_train[i]) ] if Mapper=='SE' else [ i for i in range(len(y_train)) if is_clean_service( y_train[i]) ]
x_train= [ x_train[i] for i in toKeep ]
y_train= [ y_train[i] for i in toKeep ]


In [ ]:
for j in range(len(x_test)):
    toChange= [ i for i in range(len(y_test[j])) if is_clean_service( y_test[j][i], False) ]
    y_test[j] = [ (y_test[j][i] if i not in toChange else np.array( ['none'])) for i in range(len(y_test[j])) ]

In [ ]:
# classes.remove('read_attr_-_raw-XXX')
# classes.remove('on/off-XXX')
# classes.remove('raw-XXX')
# classes.remove('lock-unlocked')
# classes.remove('lock-locked')


# classes.remove('read_attr_-_raw')
# classes.remove('on/off')
# classes.remove('raw')
classes.remove('unknown')

# classes.remove('lock')
# # classes.remove('lock')


# classes.remove('switch-on')



service_classes= ["","",""]


In [12]:
def pre_process_raw( x_data,y_data, dim_size = 128, zero_pad = False, normalize = False ,classes=None, twoD= False ):
#  y data 
    if classes is None:
        classes  = sorted(list(np.unique(  np.concatenate( y_data  ))))
    else :
        classes = sorted(classes)
    y_data_categorical = []  

    for x in y_data:
        temp = np.zeros( len(classes) )
        for y in x : 
            if y in classes:
                temp[ classes.index( y ) ] = 1
        y_data_categorical.append( temp )
    y_data_categorical = np.vstack(y_data_categorical)

#     x_data = np.array( x_data) / 1500.0
    
    x_data_temp = [] 
    
    if not zero_pad:
        if twoD:
            for x in x_data:
                temp = [] #list(x)
                lst = list(x)
                while dim_size**2 - len(temp )   > len(lst):
                    temp.extend(lst)

                while len(temp) < dim_size**2:
                    temp.append( 0 )

                x_data_temp.append(np.array( temp).reshape(dim_size,dim_size))


            x_data_temp = np.array( x_data_temp )
            x_data_temp=x_data_temp.reshape(x_data_temp.shape+(1,))
        else: 
            temp = [] 
            lst = list(x)
            for x in x_data:
                temp = [] #list(x)
                lst = list(x)
                while dim_size - len(temp )   > len(lst):
                    temp.extend(lst)

                while len(temp) < dim_size:
                    temp.append( 0 )
                
                x_data_temp.append(np.array( temp))
            
    else :
        x_data_temp = sequence.pad_sequences(x_data, maxlen=dim_size)
    
    
    if normalize:
        x_data_temp = np.array( x_data_temp) / 1500.0
    else :
        x_data_temp = np.array(x_data_temp)
    
    
    return x_data_temp ,y_data_categorical , classes

# Model Evaluation Methods

In [13]:
def make_recall_shit( inp ):
    tp = inp[1][1]
    tn = inp[0][0]
    fp = inp[0][1] 
    fn = inp[1][0]
    
    acc = (tp+tn)*1.0 / ( tp+tn+fp+fn)*1.0
    recall = tp*1.0/ ( tp+fn ) *1.0
    prec = tp*1.0 / ( tp+fp )*1.0
    
#     F= 2.0*( prec* recall )/ (prec+recall)
    F= 2.0*( tp)/ (2*tp + fp + fn)
    
    return acc, recall, prec, F

def acc_match( true, pred ):
    """
    returns exact mathc accuracy
    """
 
    return (len( [ x  for x  in  [np.sum(np.abs( true[i]- pred[i] )) for i in range(len(true))] if x  == 0]))*1.0 / len(true)


# def acc_none_zero ( true, pred ):
    

def acc_match_wierd( true, pred ):
    """
    returns exact mathc accuracy
    """
    level = 6 
    switch = 11
    threeAxis=13
    accel = 0 
    status=10
    contact=5
    
    counter  = 0 
    for i in range( len (true) ):
        if np.sum(np.abs( true[i]- pred[i] ))==0 :
            counter+=1
        else : 
            t_rec = np.array(list( pred[i]))
            
            if true[i][level]==1 and true[i][switch]==1 and t_rec[level]==1 :
                t_rec[switch]=1
            
            if true[i][threeAxis]==1 and true[i][accel]==1 and t_rec[threeAxis]==1:
                t_rec[accel] =1
            
            if true[i][status]==1 and true[i][contact]==1 and t_rec[status]==1:
                t_rec[contact]=1
#             print(t_rec , true[i])    
            if np.sum(np.abs( true[i]- t_rec ))==0 :
                counter+=1   
            
             
            
    
    return counter*1.0 / len(true)


def print_info(y_test, pred , classes , confidance=0.5 ):
    
    counts = np.sum( y_test.astype(int) , axis=0)
    
    pred[pred>=confidance] = 1
    pred[pred<confidance] = 0
    
#     acc_wierd  =acc_match_wierd(y_test, pred)
    
    conf= multilabel_confusion_matrix( y_test , pred.astype(int), labels= range(len(classes)))
    accs = [make_recall_shit(x) for x in conf]
    print( "%30s  %8s   %8s  %8s  %8s %8s %16s"  %( "Class","Accuracy", "Recall","Precision","F Score" , "Count", "TP/TN/FP/FN"))
    print( "------------------------------------------------------------------------" )
    
    for index in range(len(classes)):
        tp = conf[index][1][1]
        tn = conf[index][0][0]
        fp = conf[index][0][1] 
        fn = conf[index][1][0]
        print( "%30s  %8.3f   %8.3f  %8.3f  %8.3f  %8d %d/%d/%d/%d"  %
             (classes[index],
              accs[index][0],
              accs[index][1],
              accs[index][2],
              accs[index][3],
              counts[index],
                  tp ,
                tn ,
                fp ,
                fn ))
    n_zeros_true = len([ x  for x  in  [np.sum(np.abs( y_test[i] )) for i in range(len(y_test))] if x  == 0]  )
    n_zeros_pred = len([ x  for x  in  [np.sum(np.abs( pred[i] )) for i in range(len(pred))] if x  == 0]  )
    
    accs = np.nan_to_num(accs)
    
    print ("------------------------------------------------------------------------")
    print( "%30s  %8.3f   %8.3f  %8.3f  %8.3f  %8d %d/%d/%d/%d"  %
             ("AVERAGES",
              np.average( accs, axis=0)[0],
              np.average( accs, axis=0)[1],
              np.average( accs, axis=0)[2],
              np.average( accs, axis=0)[3],
              len(y_test),
                  0 ,
                0,
                0 ,
                0 ))
    
    print ( "Exact Match ACC : %.5f " % acc_match( y_test, pred )  )
#     print ( "Wierd Exact Match ACC : %.5f" % acc_wierd)
    print ( "Total Records : %d " % len(y_test)  )
    print ( "Total ZXeros in True : %d (%.3f)%%" % (n_zeros_true ,  n_zeros_true * 1.0/ len(y_test)  ))
    print ( "Total ZXeros in Test : %d (%.3f)%%" % (n_zeros_pred ,  n_zeros_pred * 1.0/ len(y_test)  ) )
    print ('=============================================================================')
    

In [14]:
def make_readable_results ( inp , classes , conffidance=True):
    ret = [] 
    inp =inp.astype(int)
    for xx in range(len(inp)) :
        u = inp[xx]
        temp = []
        for j in range(len(u)) : 
            if u[j] >0:
                temp.append(classes[j])
        ret.append(temp)
    return ret


def makeReadable( model , data, gt, path , classes, x, confidance=0.7):
    pred_temp = model.predict(data)
    print_info(gt, pred_temp, classes , confidance=confidance)
    print( len(classes ), len( pred_temp[0] ) )
    xcc= make_readable_results(pred_temp , classes)
    y_gt = make_readable_results( gt, classes )
    temp_dic = {} 
    for pick in range(len(xcc)): 
        temp_dic[ pick +1 ] =  { 'seq': str(data[pick]),
                               'pred': xcc[pick],
                                'true':y_gt[pick]
                               }   

    with open(path , 'w') as f:
        json.dump(temp_dic , f, indent=4)


# def makeReadable( model , data, gt, path , classes, confidance=0.7, x):
#     pred_temp = model.predict( data)
#     print_info(gt, pred_temp, classes , confidance=confidance)
#     xcc= make_readable_results( pred_temp , classes )
#     temp_dic = {} 
#     for pick in range(len(xcc)): 
#         temp_dic[ pick +1 ] = xcc[pick]  

#     with open(path , 'w') as f:
#         json.dump(temp_dic , f, indent=4)

# Create the Model 

# Calcualte per class accuracy

# Random Forest baseline calc

In [79]:
dim_size= 50
x_random_forest_train,y_random_forest_train, _ = pre_process_raw( x_train, y_train , dim_size, zero_pad=True, normalize=False, classes=classes)
# x_random_forest_test,y_random_forest_test, _ = pre_process_raw( x_test[0], y_test[0] , dim_size, zero_pad=True, normalize=False, classes=classes)
rf_tests  = [ pre_process_raw( x_test[i], y_test[i] , dim_size, zero_pad=True, normalize=False, classes=classes) for i in range(len(x_test)) ] 
# x,y, classes = pre_process_raw( x_data, y_data , dim_size, zero_pad=True, normalize=True)

In [80]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=960, max_depth=9050,
                             random_state=0 )
t_hist = clf.fit(x_random_forest_train, y_random_forest_train)




# print(clf.feature_importances_)

# print(clf.predict([[0, 0, 0, 0]]))
# from sklearn import metrics
# scores = cross_val_score(clf, x_random_forest_train, y_random_forest_train, cv=10, scoring='accuracy')

In [81]:
for i in range(len(rf_tests)) :
    print( "==================HOME Case : %s =============" % test_names[ i] )
    rf_pred= clf.predict( rf_tests[i][0])
    print_info( rf_tests[i][1], rf_pred, classes)

==================HOME Case : home_muhammed_final.json =============


/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  
/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


                         Class  Accuracy     Recall  Precision   F Score    Count      TP/TN/FP/FN
------------------------------------------------------------------------
                  acceleration     1.000      0.500     0.750     0.600         6 3/32062/1/3
                      activity     0.999      0.729     0.854     0.787        48 35/32015/6/13
                       battery     1.000        nan     0.000     0.000         0 0/32067/2/0
                        button     1.000      0.857     0.857     0.857         7 6/32061/1/1
              colorTemperature     1.000        nan       nan       nan         0 0/32069/0/0
                       contact     0.999      0.718     0.862     0.783        78 56/31982/9/22
                         level     0.995      0.815     0.128     0.221        27 22/31892/150/5
                          lock     0.999      0.786     0.229     0.355        14 11/32018/37/3
                        motion     0.971      0.183     0.824     0

/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


                         Class  Accuracy     Recall  Precision   F Score    Count      TP/TN/FP/FN
------------------------------------------------------------------------
                  acceleration     0.998      0.281     0.842     0.421        57 16/19908/3/41
                      activity     0.999      0.684     0.684     0.684        19 13/19943/6/6
                       battery     0.999      0.000     0.000     0.000         7 0/19958/3/7
                        button     0.999      0.000     0.000     0.000        14 0/19953/1/14
              colorTemperature     1.000      0.667     1.000     0.800         6 4/19962/0/2
                       contact     0.997      0.665     0.921     0.772       176 117/19782/10/59
                         level     0.986      0.704     0.064     0.117        27 19/19661/280/8
                          lock     0.997      0.771     0.380     0.509        35 27/19889/44/8
                        motion     0.981      0.194     0.483  

/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  
/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


                         Class  Accuracy     Recall  Precision   F Score    Count      TP/TN/FP/FN
------------------------------------------------------------------------
                  acceleration     0.998      0.200     1.000     0.333        25 5/9084/0/20
                      activity     0.998      0.381     0.800     0.516        21 8/9086/2/13
                       battery     1.000      0.000       nan     0.000         4 0/9105/0/4
                        button     1.000      0.000     0.000     0.000         3 0/9105/1/3
              colorTemperature     1.000        nan       nan       nan         0 0/9109/0/0
                       contact     0.996      0.615     0.923     0.738        78 48/9027/4/30
                         level     0.986      0.750     0.066     0.122        12 9/8970/127/3
                          lock     0.997      0.545     0.400     0.462        22 12/9069/18/10
                        motion     0.976      0.170     0.507     0.254    

/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  
/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


In [ ]:
# makeReadable( data=rf_tests[0][0], model=clf, classes=classes, confidance=0.7,gt=rf_tests[0][1], path='sk_home_out.json' )

In [ ]:
# print_info(y_random_forest_test, rf_pred, classes)

In [ ]:
# print_info(y_random_forest_test, rf_pred, classes)

In [ ]:
# print ( "mean : %f \nstd: %f\nmax:%f" %( scores.mean(), scores.std(), scores.max()) )

# LSTM 

In [ ]:
# cnts  = np.unique(np.array([ len(x) for x  in x_train ]), return_counts=True)
# # np.sort( cnts[1] )
# cnts

In [ ]:
test_names

In [51]:
# create the model
dim_size =15
x_lstm_prossed_train,y_lstm_prossed_train, _ = pre_process_raw( x_train, y_train , dim_size, zero_pad=False, normalize=False,classes=classes)
_, y_s_lstm_processed_train ,_ =  pre_process_raw( x_train, y_train_service , dim_size, zero_pad=False, normalize=False,classes=service_classes)
# x_lstm_prossed_test,y_lstm_prossed_test, _ = pre_process_raw( x_test, y_test_2 , dim_size, zero_pad=True, normalize=False,classes=classes)
lstm_tests  = [ pre_process_raw( x_test[i], y_test[i] , dim_size, zero_pad=False, normalize=False, classes=classes) for i in range(len(x_test)) ] 
lstm_tests_services  = [ pre_process_raw( x_test[i], y_test_service[i] , dim_size, zero_pad=False, normalize=False, classes=service_classes) for i in range(len(x_test)) ] 



In [52]:
x_lstm_prossed_train[0]

array([290,  99, 412,  99, 290,  99, 412,  99, 290,  99, 412,  99,   0,
         0,   0])

In [53]:
for i in range( len(lstm_tests) ):
    print( len( lstm_tests[i][0] ), len( lstm_tests_services[i][0] ) )

32069 32069
19968 19968
9109 9109
6404 6404


In [54]:
# x_lstm_prossed_test2 = np.expand_dims(x_lstm_prossed_test,axis=1)
# x_lstm_prossed_train2 =np.expand_dims(x_lstm_prossed_train,axis=1)

for tt  in range( len(lstm_tests) ):
    lstm_tests[tt]= (lstm_tests[tt][0].reshape(len(lstm_tests[tt][0]),dim_size,1) , lstm_tests[tt][1],lstm_tests_services[tt][1] )
# x_lstm_prossed_test2 = x_lstm_prossed_test.reshape(len(x_lstm_prossed_test),dim_size,1)
x_lstm_prossed_train2 =x_lstm_prossed_train.reshape(len(x_lstm_prossed_train),dim_size,1)

# y_lstm_prossed_test2 = y_lstm_prossed_test.reshape(len(y_lstm_prossed_test),len(classes),1)
# y_lstm_prossed_train2 =y_lstm_prossed_train.reshape(len(y_lstm_prossed_train),len(classes),1)

In [55]:
np.sum(y_s_lstm_processed_train)

60562.0

In [57]:
## Bidirectional LSTM :D 
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers import Flatten


import numpy as np
import os
import tensorflow as tf
from keras.models import Model

from keras.layers import Input, merge, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D, Dropout, Conv2DTranspose, UpSampling2D, Lambda
from keras.layers import Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization as bn
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.optimizers import RMSprop
from keras import regularizers
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers.merge import add
from keras.models import load_model
import numpy as np
from keras.regularizers import l2
from keras.layers import Conv1D
from keras.layers.merge import add
from keras.layers import Dense,Conv1D,Dropout,Activation,BatchNormalization,MaxPooling1D,Flatten,Masking,TimeDistributed
from keras.layers.recurrent import LSTM,GRU,SimpleRNN
from keras.models import Input,Sequential,Model
from keras.layers.merge import add
from keras.optimizers import Adam
from keras.losses import MSE,MSLE

model2 = Sequential()


def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss


import tensorflow as tf

def f1_perRow(y_true, y_pred):
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)






def fn_loss ( y_true, y_pred ):
#     tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=1)
#     tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=1)
#     fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=1)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    loss = fp
    loss = tf.where(tf.is_nan(loss), tf.zeros_like(loss), loss)
    return 1.0 - K.mean(loss)



def fn_rate ( y_true, y_pred ):
#     tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=1)
#     tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=1)
#     fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=1)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    loss = fn
    loss = tf.where(tf.is_nan(loss), tf.zeros_like(loss), loss)
    return  K.mean(loss)



def fp_loss ( y_true, y_pred ):
#     tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=1)
#     tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=1)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=1)
#     fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=1)

    loss = fp
    loss = tf.where(tf.is_nan(loss), tf.zeros_like(loss), loss)
    return  K.mean(loss)


def fp_loss_perRow ( y_true, y_pred ):
#     tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=1)
#     tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=1)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
#     fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=1)

    loss = fp
    loss = tf.where(tf.is_nan(loss), tf.zeros_like(loss), loss)
    return  K.sum(loss)



def fp_rate ( y_true, y_pred ):
#     tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=1)
#     tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=1)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=1)
#     fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=1)

    loss = fp
    loss = tf.where(tf.is_nan(loss), tf.zeros_like(loss), loss)
    return  K.mean(loss)





def f1_perClass(y_true, y_pred):

    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=1)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=1)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=1)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=1)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss_perClass(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=1)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=1)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=1)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=1)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return (1 - K.mean(f1))**2

def f1_loss_perRow(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return (1 - K.mean(f1))**2



inputs  = Input( ( dim_size,1 ) )


out = Conv1D(128,3,padding='same')(inputs)
out = BatchNormalization()(out)
out = Activation('relu')(out)
out = Dropout(0.2)(out)
out = Conv1D(128,3,padding='same')(out)
# bi_d_1 = Bidirectional( LSTM(200 ,  recurrent_dropout=0.12, return_sequences=True),input_shape=[dim_size,1],merge_mode='concat') (inputs)
lstm_1 =  LSTM(100 ,  recurrent_dropout=0.04, return_sequences=True)(out)
# lstm_2 = LSTM(30 ,  recurrent_dropout=0.14, return_sequences=True)(lstm_1)

bi_d_1 =Dense(128, activation='relu')  (lstm_1)
bi_d_1= Dropout( 0.1 )(bi_d_1)
lstm_1 =  Dense(128, activation='relu')(bi_d_1)
lstm_1 = Dropout(0.1)(lstm_1)
lstm_2 = Dense(128, activation='relu')(lstm_1)



# td_1    = TimeDistributed(Dense(256, activation='relu'))(lstm_2)
# dout_1  = Dropout(0.1)(td_1)
dout_1  = Dropout(0.1)(lstm_2)
flt_1   = Flatten()(dout_1)
dense_1 = Dense(128, activation='relu')(flt_1)
dout_2  = Dropout(0.2)(dense_1)











out = Conv1D(128,2,padding='same')(inputs)
out = BatchNormalization()(out)
out = Activation('relu')(out)
out = Dropout(0.2)(out)
out = Conv1D(128,2,padding='same')(out)
# bi_d_1 = Bidirectional( LSTM(200 ,  recurrent_dropout=0.12, return_sequences=True),input_shape=[dim_size,1],merge_mode='concat') (inputs)
lstm_1 =  LSTM(100 ,  recurrent_dropout=0.04, return_sequences=True)(out)
# lstm_2 = LSTM(30 ,  recurrent_dropout=0.14, return_sequences=True)(lstm_1)

bi_d_1 =Dense(128, activation='relu')  (lstm_1)
bi_d_1= Dropout( 0.1 )(bi_d_1)
lstm_1 =  Dense(128, activation='relu')(bi_d_1)
lstm_1 = Dropout(0.1)(lstm_1)
lstm_2 = Dense(128, activation='relu')(lstm_1)



# td_1    = TimeDistributed(Dense(256, activation='relu'))(lstm_2)
# dout_1  = Dropout(0.1)(td_1)
dout_1  = Dropout(0.1)(lstm_2)
flt_1   = Flatten()(dout_1)
dense_1 = Dense(128, activation='relu')(flt_1)
dout_2_2  = Dropout(0.2)(dense_1)









# bi_d_1 = Bidirectional( LSTM(200 ,  recurrent_dropout=0.12, return_sequences=True),input_shape=[dim_size,1],merge_mode='concat') (inputs)
lstm_1 =  LSTM(100 ,  recurrent_dropout=0.04, return_sequences=True)(inputs)
# lstm_1 = LSTM(40 ,  recurrent_dropout=0.14, return_sequences=True)(lstm_1)

bi_d_raw_1 =Dense(128, activation='relu')  (lstm_1)
bi_d_raw_1 = Dropout(0.1)(bi_d_raw_1)
lstm_raw_1 =  Dense(128, activation='relu')(bi_d_raw_1)
lstm_raw_1= Dropout(0.1)(lstm_raw_1)
lstm_raw_2 = Dense(128, activation='relu')(lstm_raw_1)

dout_1  = Dropout(0.1)(lstm_raw_2)
flt_1   = Flatten()(dout_1)
dense_1 = Dense(128, activation='relu')(flt_1)
dout_3  = Dropout(0.2)(dense_1)





out = Conv1D(128,3,padding='same')(inputs)
out = BatchNormalization()(out)
out = Activation('relu')(out)
out = Dropout(0.2)(out)
out = Conv1D(128,3,padding='same')(out)
# out = Flatten()(out)
# out = MaxPooling1D(2,padding='same', name ='pooling')(out)


out = Conv1D(128,3,padding='same')(out)
out = BatchNormalization()(out)
out = Activation('relu')(out)
out = Dropout(0.2)(out)
out = Conv1D(128,3,padding='same')(out)
# out = Flatten()(out)
# out = MaxPooling1D(2,padding='same', name ='pooling2')(out)


out = Conv1D(128,3,padding='same')(out)
out = BatchNormalization()(out)
out = Activation('relu')(out)
out = Dropout(0.2)(out)
out = Conv1D(128,3,padding='same')(out)
# out = Flatten()(out)
# out = MaxPooling1D(2,padding='same', name ='pooling')(out)



# fl_out_1 = Flatten()(dout_2)

fl_out_cnn = Flatten()(out)

# out_new = concatenate( [fl_out_1, fl_out_cnn] , name='mergerguy')
out_new = concatenate( [dout_2, fl_out_cnn,dout_3,dout_2_2] , name='mergerguy')

dens_out_1 = Dense( 128, activation='relu' )(out_new)
dens_out_1= Dropout(0.1)(dens_out_1)
dens_out_2 = Dense( 128, activation='relu' )(dens_out_1)
dens_out_2=Dropout(0.1)(dens_out_2)
dens_out_3 = Dense( 128, activation='relu' )(dens_out_2)

# fl2  = Flatten()(out_new)

out_put_final = Dense(len(classes), activation='sigmoid', name='Event_output')(dens_out_3)

toService_1 = Dense( 130, name="to_service1" )(dens_out_3)
toService_1 = Dense( 130, name="to_service2" )(toService_1)

service_output = Dense(len(classes  ), activation="sigmoid", name = 'service_output')(toService_1)



model2 = Model(inputs=[inputs], outputs=[service_output])




# model2.add(Bidirectional( LSTM(200 ,  recurrent_dropout=0.12, return_sequences=True),input_shape=[dim_size,1],merge_mode='concat'))
# model2.add( LSTM(60 ,  recurrent_dropout=0.04, return_sequences=True))
# model2.add( LSTM(30 ,  recurrent_dropout=0.14, return_sequences=True))
# # model2.add(Bidirectional( LSTM(100 ,  recurrent_dropout=0.04, return_sequences=True),merge_mode='concat'))
# model2.add(TimeDistributed(Dense(256, activation='relu')))
# model2.add(Dropout(0.1))
# model2.add(Flatten())
# model2.add(Dense(128, activation='relu'))
# model2.add(Dropout(0.2))
# model2.add(Dense(len(classes), activation='sigmoid'))

# model2.compile(loss=weighted_categorical_crossentropy(weights=weights), optimizer='adam', metrics=['accuracy'])



losses = {
#     "service_output": f1_loss_perClass ,
    "service_output": f1_loss_perRow ,
    "service_output": fp_loss, 
    "service_output": "categorical_crossentropy",
}
lossWeights = {#"service_output": 20,
               "service_output": 30.0 ,
    "service_output": 25,
    "service_output": 20
}
 


model2.compile(loss=losses,loss_weights=lossWeights, optimizer='adam', metrics=[fp_rate, f1_perRow,f1_perClass,'acc'])
# model2.compile(loss=losses, loss_weights=lossWeights, optimizer='adam', metrics=['accuracy'])

# checkpoint = ModelCheckpoint('IoTDownNet', monitor='loss', verbose=0, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]

print(model2.summary())
hist2 = model2.fit(x_lstm_prossed_train2, y_lstm_prossed_train, epochs=100, batch_size=11500, shuffle=True)





____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_6 (InputLayer)             (None, 15, 1)         0                                            
____________________________________________________________________________________________________
conv1d_53 (Conv1D)               (None, 15, 128)       512         input_6[0][0]                    
____________________________________________________________________________________________________
conv1d_49 (Conv1D)               (None, 15, 128)       512         input_6[0][0]                    
____________________________________________________________________________________________________
batch_normalization_27 (BatchNor (None, 15, 128)       512         conv1d_53[0][0]                  
___________________________________________________________________________________________

Epoch 1/100
65266/65266 [==============================] - 6s - loss: 46.8733 - fp_rate: 4.6743 - f1_perRow: 0.1077 - f1_perClass: 0.2969 - acc: 0.4831     
Epoch 2/100
65266/65266 [==============================] - 3s - loss: 33.0607 - fp_rate: 1.9123 - f1_perRow: 0.1561 - f1_perClass: 0.4923 - acc: 0.4900     
Epoch 3/100
65266/65266 [==============================] - 3s - loss: 30.4007 - fp_rate: 1.5741 - f1_perRow: 0.1911 - f1_perClass: 0.5279 - acc: 0.4140     
Epoch 4/100
65266/65266 [==============================] - 3s - loss: 27.2212 - fp_rate: 1.1292 - f1_perRow: 0.2269 - f1_perClass: 0.5969 - acc: 0.4139     
Epoch 5/100
65266/65266 [==============================] - 3s - loss: 26.1174 - fp_rate: 1.1373 - f1_perRow: 0.2545 - f1_perClass: 0.6117 - acc: 0.4139     
Epoch 6/100
65266/65266 [==============================] - 3s - loss: 25.4940 - fp_rate: 1.1354 - f1_perRow: 0.2683 - f1_perClass: 0.6180 - acc: 0.4224     
Epoch 7/100
65266/65266 [==============================] -

In [83]:

def fp_loss_perRow ( y_true, y_pred ):
#     tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=1)
#     tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=1)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
#     fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=1)

    loss = fp
    loss = tf.where(tf.is_nan(loss), tf.zeros_like(loss), loss)
    return  K.sum(loss)

losses = {
#     "service_output": f1_loss_perClass ,
    "service_output": f1_loss_perClass ,
    "service_output": fp_loss_perRow ,
    "service_output": "categorical_crossentropy",
}
lossWeights = {#"service_output": 20,
               "service_output": 30.0 ,
            "service_output": 30.0 ,
    "service_output": 20
}
 


model2.compile(loss=losses, loss_weights=lossWeights, optimizer=keras.optimizers.Adam(lr=1e-5  ), metrics=[f1_perRow,f1_perClass,'acc'])


In [84]:
# model2.compile(loss=losses,loss_weights=lossWeights, optimizer=keras.optimizers.Adam(lr=8e-5  ), metrics=[fp_rate,f1_perRow,f1_perClass,'acc'])
hist2 = model2.fit(x_lstm_prossed_train2, y_lstm_prossed_train, epochs=300, batch_size=11500, shuffle=True)


Epoch 1/300
65266/65266 [==============================] - 6s - loss: 11.6022 - f1_perRow: 0.5678 - f1_perClass: 0.8576 - acc: 0.8461     
Epoch 2/300
65266/65266 [==============================] - 3s - loss: 11.2201 - f1_perRow: 0.5786 - f1_perClass: 0.8604 - acc: 0.8472     
Epoch 3/300
65266/65266 [==============================] - 3s - loss: 11.3462 - f1_perRow: 0.5785 - f1_perClass: 0.8591 - acc: 0.8471     
Epoch 4/300
65266/65266 [==============================] - 3s - loss: 11.5365 - f1_perRow: 0.5712 - f1_perClass: 0.8576 - acc: 0.8463     
Epoch 5/300
65266/65266 [==============================] - 3s - loss: 11.2033 - f1_perRow: 0.5863 - f1_perClass: 0.8602 - acc: 0.8463     
Epoch 6/300
65266/65266 [==============================] - 3s - loss: 11.4254 - f1_perRow: 0.5715 - f1_perClass: 0.8582 - acc: 0.8461     
Epoch 7/300
65266/65266 [==============================] - 3s - loss: 11.2435 - f1_perRow: 0.5784 - f1_perClass: 0.8598 - acc: 0.8459     
Epoch 8/300
65266/65266 [==

65266/65266 [==============================] - 3s - loss: 11.3160 - f1_perRow: 0.5862 - f1_perClass: 0.8596 - acc: 0.8467     
Epoch 60/300
65266/65266 [==============================] - 3s - loss: 11.1736 - f1_perRow: 0.5866 - f1_perClass: 0.8606 - acc: 0.8468     
Epoch 61/300
65266/65266 [==============================] - 3s - loss: 11.1865 - f1_perRow: 0.5855 - f1_perClass: 0.8605 - acc: 0.8472     
Epoch 62/300
65266/65266 [==============================] - 3s - loss: 11.2233 - f1_perRow: 0.5845 - f1_perClass: 0.8602 - acc: 0.8468     
Epoch 63/300
65266/65266 [==============================] - 3s - loss: 11.5007 - f1_perRow: 0.5693 - f1_perClass: 0.8579 - acc: 0.8460     
Epoch 64/300
65266/65266 [==============================] - 3s - loss: 11.2869 - f1_perRow: 0.5773 - f1_perClass: 0.8597 - acc: 0.8464     
Epoch 65/300
65266/65266 [==============================] - 3s - loss: 11.2467 - f1_perRow: 0.5856 - f1_perClass: 0.8600 - acc: 0.8467     
Epoch 66/300
65266/65266 [=======

65266/65266 [==============================] - 3s - loss: 11.4826 - f1_perRow: 0.5755 - f1_perClass: 0.8580 - acc: 0.8465     
Epoch 118/300
65266/65266 [==============================] - 3s - loss: 11.1616 - f1_perRow: 0.5846 - f1_perClass: 0.8607 - acc: 0.8469     
Epoch 119/300
65266/65266 [==============================] - 3s - loss: 11.3410 - f1_perRow: 0.5810 - f1_perClass: 0.8595 - acc: 0.8469     
Epoch 120/300
65266/65266 [==============================] - 3s - loss: 11.2857 - f1_perRow: 0.5804 - f1_perClass: 0.8599 - acc: 0.8466     
Epoch 121/300
65266/65266 [==============================] - 3s - loss: 11.1310 - f1_perRow: 0.5860 - f1_perClass: 0.8609 - acc: 0.8464     
Epoch 122/300
65266/65266 [==============================] - 3s - loss: 11.2573 - f1_perRow: 0.5828 - f1_perClass: 0.8601 - acc: 0.8478     
Epoch 123/300
65266/65266 [==============================] - 3s - loss: 11.1093 - f1_perRow: 0.5834 - f1_perClass: 0.8611 - acc: 0.8483     
Epoch 124/300
65266/65266 [

65266/65266 [==============================] - 3s - loss: 11.4461 - f1_perRow: 0.5749 - f1_perClass: 0.8588 - acc: 0.8471     
Epoch 176/300
65266/65266 [==============================] - 3s - loss: 11.2843 - f1_perRow: 0.5766 - f1_perClass: 0.8599 - acc: 0.8472     
Epoch 177/300
65266/65266 [==============================] - 3s - loss: 11.3639 - f1_perRow: 0.5820 - f1_perClass: 0.8593 - acc: 0.8477     
Epoch 178/300
65266/65266 [==============================] - 3s - loss: 11.3022 - f1_perRow: 0.5838 - f1_perClass: 0.8597 - acc: 0.8478     
Epoch 179/300
65266/65266 [==============================] - 3s - loss: 11.3261 - f1_perRow: 0.5848 - f1_perClass: 0.8598 - acc: 0.8483     
Epoch 180/300
65266/65266 [==============================] - 3s - loss: 11.3033 - f1_perRow: 0.5778 - f1_perClass: 0.8597 - acc: 0.8469     
Epoch 181/300
65266/65266 [==============================] - 3s - loss: 11.2629 - f1_perRow: 0.5845 - f1_perClass: 0.8597 - acc: 0.8474     
Epoch 182/300
65266/65266 [

65266/65266 [==============================] - 3s - loss: 11.3906 - f1_perRow: 0.5813 - f1_perClass: 0.8590 - acc: 0.8474     
Epoch 234/300
65266/65266 [==============================] - 3s - loss: 11.2835 - f1_perRow: 0.5805 - f1_perClass: 0.8599 - acc: 0.8479     
Epoch 235/300
65266/65266 [==============================] - 3s - loss: 12.1028 - f1_perRow: 0.5612 - f1_perClass: 0.8545 - acc: 0.8467     
Epoch 236/300
65266/65266 [==============================] - 3s - loss: 11.4233 - f1_perRow: 0.5841 - f1_perClass: 0.8590 - acc: 0.8479     
Epoch 237/300
65266/65266 [==============================] - 3s - loss: 11.3429 - f1_perRow: 0.5815 - f1_perClass: 0.8591 - acc: 0.8471     
Epoch 238/300
65266/65266 [==============================] - 3s - loss: 11.2397 - f1_perRow: 0.5781 - f1_perClass: 0.8602 - acc: 0.8479     
Epoch 239/300
65266/65266 [==============================] - 3s - loss: 11.1851 - f1_perRow: 0.5896 - f1_perClass: 0.8607 - acc: 0.8475     
Epoch 240/300
65266/65266 [

65266/65266 [==============================] - 3s - loss: 11.3208 - f1_perRow: 0.5708 - f1_perClass: 0.8598 - acc: 0.8468     
Epoch 292/300
65266/65266 [==============================] - 3s - loss: 11.4094 - f1_perRow: 0.5800 - f1_perClass: 0.8593 - acc: 0.8472     
Epoch 293/300
65266/65266 [==============================] - 3s - loss: 11.3501 - f1_perRow: 0.5807 - f1_perClass: 0.8600 - acc: 0.8479     
Epoch 294/300
65266/65266 [==============================] - 3s - loss: 11.4239 - f1_perRow: 0.5808 - f1_perClass: 0.8588 - acc: 0.8470     
Epoch 295/300
65266/65266 [==============================] - 3s - loss: 11.3322 - f1_perRow: 0.5757 - f1_perClass: 0.8600 - acc: 0.8480     
Epoch 296/300
65266/65266 [==============================] - 3s - loss: 11.4332 - f1_perRow: 0.5789 - f1_perClass: 0.8591 - acc: 0.8472     
Epoch 297/300
65266/65266 [==============================] - 3s - loss: 11.6220 - f1_perRow: 0.5723 - f1_perClass: 0.8578 - acc: 0.8471     
Epoch 298/300
65266/65266 [

In [ ]:
model2.compile(loss=losses, optimizer=keras.optimizers.Adam(lr=1e-8  ), metrics=[f1_perRow,f1_perClass,'acc'])
hist2 = model2.fit(x_lstm_prossed_train2, y_lstm_prossed_train, epochs=3300, batch_size=11500, shuffle=True)

Epoch 1/3300
65266/65266 [==============================] - 7s - loss: 0.5572 - f1_perRow: 0.5908 - f1_perClass: 0.8610 - acc: 0.8483     
Epoch 2/3300
65266/65266 [==============================] - 3s - loss: 0.5691 - f1_perRow: 0.5773 - f1_perClass: 0.8590 - acc: 0.8483     
Epoch 3/3300
65266/65266 [==============================] - 3s - loss: 0.5618 - f1_perRow: 0.5863 - f1_perClass: 0.8604 - acc: 0.8482     
Epoch 4/3300
65266/65266 [==============================] - 3s - loss: 0.5688 - f1_perRow: 0.5873 - f1_perClass: 0.8595 - acc: 0.8478     
Epoch 5/3300
65266/65266 [==============================] - 3s - loss: 0.5629 - f1_perRow: 0.5887 - f1_perClass: 0.8602 - acc: 0.8484     
Epoch 6/3300
65266/65266 [==============================] - 3s - loss: 0.5886 - f1_perRow: 0.5705 - f1_perClass: 0.8564 - acc: 0.8482     
Epoch 7/3300
65266/65266 [==============================] - 3s - loss: 0.5617 - f1_perRow: 0.5813 - f1_perClass: 0.8606 - acc: 0.8483     
Epoch 8/3300
65266/65266 [=

65266/65266 [==============================] - 3s - loss: 0.5776 - f1_perRow: 0.5699 - f1_perClass: 0.8578 - acc: 0.8475     
Epoch 60/3300
65266/65266 [==============================] - 3s - loss: 0.5701 - f1_perRow: 0.5768 - f1_perClass: 0.8594 - acc: 0.8487     
Epoch 61/3300
65266/65266 [==============================] - 3s - loss: 0.5750 - f1_perRow: 0.5767 - f1_perClass: 0.8585 - acc: 0.8476     
Epoch 62/3300
65266/65266 [==============================] - 3s - loss: 0.5599 - f1_perRow: 0.5845 - f1_perClass: 0.8603 - acc: 0.8478     
Epoch 63/3300
65266/65266 [==============================] - 3s - loss: 0.5688 - f1_perRow: 0.5841 - f1_perClass: 0.8591 - acc: 0.8476     
Epoch 64/3300
65266/65266 [==============================] - 3s - loss: 0.5666 - f1_perRow: 0.5803 - f1_perClass: 0.8597 - acc: 0.8466     
Epoch 65/3300
65266/65266 [==============================] - 3s - loss: 0.5709 - f1_perRow: 0.5796 - f1_perClass: 0.8591 - acc: 0.8479     
Epoch 66/3300
65266/65266 [=======

65266/65266 [==============================] - 3s - loss: 0.5556 - f1_perRow: 0.5913 - f1_perClass: 0.8613 - acc: 0.8482     
Epoch 118/3300
65266/65266 [==============================] - 3s - loss: 0.5748 - f1_perRow: 0.5765 - f1_perClass: 0.8585 - acc: 0.8472     
Epoch 119/3300
65266/65266 [==============================] - 3s - loss: 0.5591 - f1_perRow: 0.5874 - f1_perClass: 0.8606 - acc: 0.8479     
Epoch 120/3300
65266/65266 [==============================] - 3s - loss: 0.5706 - f1_perRow: 0.5836 - f1_perClass: 0.8591 - acc: 0.8477     
Epoch 121/3300
65266/65266 [==============================] - 3s - loss: 0.5661 - f1_perRow: 0.5886 - f1_perClass: 0.8600 - acc: 0.8484     
Epoch 122/3300
65266/65266 [==============================] - 3s - loss: 0.5580 - f1_perRow: 0.5905 - f1_perClass: 0.8609 - acc: 0.8474     
Epoch 123/3300
65266/65266 [==============================] - 3s - loss: 0.5760 - f1_perRow: 0.5704 - f1_perClass: 0.8581 - acc: 0.8473     
Epoch 124/3300
65266/65266 [

65266/65266 [==============================] - 3s - loss: 0.5581 - f1_perRow: 0.5881 - f1_perClass: 0.8607 - acc: 0.8486     
Epoch 176/3300
65266/65266 [==============================] - 3s - loss: 0.5651 - f1_perRow: 0.5837 - f1_perClass: 0.8599 - acc: 0.8474     
Epoch 177/3300
65266/65266 [==============================] - 3s - loss: 0.5708 - f1_perRow: 0.5798 - f1_perClass: 0.8591 - acc: 0.8480     
Epoch 178/3300
65266/65266 [==============================] - 3s - loss: 0.5631 - f1_perRow: 0.5880 - f1_perClass: 0.8600 - acc: 0.8472     
Epoch 179/3300
65266/65266 [==============================] - 3s - loss: 0.5759 - f1_perRow: 0.5735 - f1_perClass: 0.8581 - acc: 0.8475     
Epoch 180/3300
65266/65266 [==============================] - 3s - loss: 0.5610 - f1_perRow: 0.5843 - f1_perClass: 0.8604 - acc: 0.8477     
Epoch 181/3300
65266/65266 [==============================] - 3s - loss: 0.5636 - f1_perRow: 0.5833 - f1_perClass: 0.8602 - acc: 0.8486     
Epoch 182/3300
65266/65266 [

65266/65266 [==============================] - 3s - loss: 0.5614 - f1_perRow: 0.5897 - f1_perClass: 0.8607 - acc: 0.8483     
Epoch 234/3300
65266/65266 [==============================] - 3s - loss: 0.5682 - f1_perRow: 0.5730 - f1_perClass: 0.8591 - acc: 0.8473     
Epoch 235/3300
65266/65266 [==============================] - 3s - loss: 0.5894 - f1_perRow: 0.5744 - f1_perClass: 0.8560 - acc: 0.8472     
Epoch 236/3300
65266/65266 [==============================] - 3s - loss: 0.5555 - f1_perRow: 0.5902 - f1_perClass: 0.8617 - acc: 0.8478     
Epoch 237/3300
65266/65266 [==============================] - 3s - loss: 0.5630 - f1_perRow: 0.5877 - f1_perClass: 0.8603 - acc: 0.8483     
Epoch 238/3300
65266/65266 [==============================] - 3s - loss: 0.5642 - f1_perRow: 0.5866 - f1_perClass: 0.8599 - acc: 0.8477     
Epoch 239/3300
65266/65266 [==============================] - 3s - loss: 0.5589 - f1_perRow: 0.5816 - f1_perClass: 0.8609 - acc: 0.8477     
Epoch 240/3300
65266/65266 [

...

65266/65266 [==============================] - 3s - loss: 0.5623 - f1_perRow: 0.5875 - f1_perClass: 0.8604 - acc: 0.8482     
Epoch 408/3300
65266/65266 [==============================] - 3s - loss: 0.5779 - f1_perRow: 0.5831 - f1_perClass: 0.8578 - acc: 0.8478     
Epoch 409/3300
65266/65266 [==============================] - 3s - loss: 0.5590 - f1_perRow: 0.5840 - f1_perClass: 0.8607 - acc: 0.8477     
Epoch 410/3300
65266/65266 [==============================] - 3s - loss: 0.5669 - f1_perRow: 0.5771 - f1_perClass: 0.8596 - acc: 0.8485     
Epoch 411/3300
65266/65266 [==============================] - 3s - loss: 0.5650 - f1_perRow: 0.5864 - f1_perClass: 0.8599 - acc: 0.8479     
Epoch 412/3300
65266/65266 [==============================] - 3s - loss: 0.5554 - f1_perRow: 0.5869 - f1_perClass: 0.8611 - acc: 0.8480     
Epoch 413/3300
65266/65266 [==============================] - 3s - loss: 0.5525 - f1_perRow: 0.5945 - f1_perClass: 0.8619 - acc: 0.8481     
Epoch 414/3300
65266/65266 [

65266/65266 [==============================] - 3s - loss: 0.5735 - f1_perRow: 0.5805 - f1_perClass: 0.8585 - acc: 0.8471     
Epoch 466/3300
65266/65266 [==============================] - 3s - loss: 0.5556 - f1_perRow: 0.5864 - f1_perClass: 0.8610 - acc: 0.8480     
Epoch 467/3300
65266/65266 [==============================] - 3s - loss: 0.5556 - f1_perRow: 0.5932 - f1_perClass: 0.8613 - acc: 0.8480     
Epoch 468/3300
65266/65266 [==============================] - 3s - loss: 0.5611 - f1_perRow: 0.5919 - f1_perClass: 0.8605 - acc: 0.8490     
Epoch 469/3300
65266/65266 [==============================] - 3s - loss: 0.5797 - f1_perRow: 0.5725 - f1_perClass: 0.8578 - acc: 0.8476     
Epoch 470/3300
65266/65266 [==============================] - 3s - loss: 0.5675 - f1_perRow: 0.5841 - f1_perClass: 0.8595 - acc: 0.8481     
Epoch 471/3300
65266/65266 [==============================] - 3s - loss: 0.5591 - f1_perRow: 0.5907 - f1_perClass: 0.8608 - acc: 0.8473     
Epoch 472/3300
65266/65266 [

65266/65266 [==============================] - 3s - loss: 0.5620 - f1_perRow: 0.5844 - f1_perClass: 0.8603 - acc: 0.8477     
Epoch 524/3300
65266/65266 [==============================] - 3s - loss: 0.5735 - f1_perRow: 0.5823 - f1_perClass: 0.8588 - acc: 0.8475     
Epoch 525/3300
65266/65266 [==============================] - 3s - loss: 0.5574 - f1_perRow: 0.5907 - f1_perClass: 0.8609 - acc: 0.8475     
Epoch 526/3300
65266/65266 [==============================] - 3s - loss: 0.5699 - f1_perRow: 0.5795 - f1_perClass: 0.8594 - acc: 0.8477     
Epoch 527/3300
65266/65266 [==============================] - 3s - loss: 0.5633 - f1_perRow: 0.5900 - f1_perClass: 0.8600 - acc: 0.8479     
Epoch 528/3300
65266/65266 [==============================] - 3s - loss: 0.5583 - f1_perRow: 0.5864 - f1_perClass: 0.8608 - acc: 0.8478     
Epoch 529/3300
65266/65266 [==============================] - 3s - loss: 0.5639 - f1_perRow: 0.5821 - f1_perClass: 0.8601 - acc: 0.8478     
Epoch 530/3300
65266/65266 [

65266/65266 [==============================] - 3s - loss: 0.5600 - f1_perRow: 0.5878 - f1_perClass: 0.8608 - acc: 0.8476     
Epoch 582/3300
65266/65266 [==============================] - 3s - loss: 0.5658 - f1_perRow: 0.5764 - f1_perClass: 0.8596 - acc: 0.8472     
Epoch 583/3300
65266/65266 [==============================] - 3s - loss: 0.5609 - f1_perRow: 0.5837 - f1_perClass: 0.8604 - acc: 0.8479     
Epoch 584/3300
65266/65266 [==============================] - 3s - loss: 0.5702 - f1_perRow: 0.5821 - f1_perClass: 0.8593 - acc: 0.8482     
Epoch 585/3300
65266/65266 [==============================] - 3s - loss: 0.5661 - f1_perRow: 0.5835 - f1_perClass: 0.8596 - acc: 0.8476     
Epoch 586/3300
65266/65266 [==============================] - 3s - loss: 0.5637 - f1_perRow: 0.5847 - f1_perClass: 0.8598 - acc: 0.8473     
Epoch 587/3300
65266/65266 [==============================] - 3s - loss: 0.5617 - f1_perRow: 0.5830 - f1_perClass: 0.8604 - acc: 0.8484     
Epoch 588/3300
65266/65266 [

65266/65266 [==============================] - 3s - loss: 0.5566 - f1_perRow: 0.5875 - f1_perClass: 0.8612 - acc: 0.8478     
Epoch 640/3300
65266/65266 [==============================] - 3s - loss: 0.5742 - f1_perRow: 0.5808 - f1_perClass: 0.8587 - acc: 0.8485     
Epoch 641/3300
65266/65266 [==============================] - 3s - loss: 0.5680 - f1_perRow: 0.5802 - f1_perClass: 0.8592 - acc: 0.8482     
Epoch 642/3300
65266/65266 [==============================] - 3s - loss: 0.5612 - f1_perRow: 0.5890 - f1_perClass: 0.8604 - acc: 0.8475     
Epoch 643/3300
65266/65266 [==============================] - 3s - loss: 0.5728 - f1_perRow: 0.5743 - f1_perClass: 0.8587 - acc: 0.8475     
Epoch 644/3300
65266/65266 [==============================] - 3s - loss: 0.5686 - f1_perRow: 0.5784 - f1_perClass: 0.8592 - acc: 0.8474     
Epoch 645/3300
65266/65266 [==============================] - 3s - loss: 0.5754 - f1_perRow: 0.5733 - f1_perClass: 0.8585 - acc: 0.8475     
Epoch 646/3300
65266/65266 [

In [ ]:
model2.compile(loss=weighted_categorical_crossentropy(weights=weights), optimizer=keras.optimizers.Adam(lr=8e-5  ), metrics=['accuracy'])

In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=8e-5  ), metrics=['accuracy'])

In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model2.compile(loss=weighted_categorical_crossentropy(weights=weights), optimizer='adam', metrics=['accuracy'])

In [ ]:
hist2 = model2.fit(x_lstm_prossed_train2, y_lstm_prossed_train, epochs=300, batch_size=3500, shuffle=True)

In [ ]:
model2.save( "LSTM-sigmoid-withRemovedClasses" )

In [ ]:
plt.plot(hist2.history['loss'], c='red')
plt.plot(hist2.history['acc'], c='blue')

In [ ]:
plt.plot(hist2.history['loss'], c='red')
# plt.plot(hist2.history['acc'], c='blue')

In [ ]:
from keras.utils import plot_model
plot_model(model2, to_file='model.png')

In [ ]:
# from keras.models import load_model
# model2=load_model( "LSTM_withSigmoid_LargeData_F1_E100_B500_MSE_False"  
#            )

In [ ]:
for i in range(len(lstm_tests)) :
    print( "==================HOME Case : %s =============" % test_names[i])
    makeReadable( classes=classes, confidance=0.7,data=lstm_tests[i][0],gt=lstm_tests[i][1],model=model2,path=test_names[i],x=lstm_tests[i][0])
    
#     lstm_pred= model2.predict( lstm_tests[i][0])
    

In [ ]:
lstm_pred= model2.predict( lstm_tests[i][0])

In [97]:
lstm_pred= model2.predict( lstm_tests[1][0])
lstm_pred__ = np.array(list(lstm_pred))

In [104]:
lstm_pred__ = np.array(list(lstm_pred))
print_info( lstm_tests[1][1], lstm_pred__, classes , confidance=0.59)

/home/omid/.conda/envs/iot_new/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


                         Class  Accuracy     Recall  Precision   F Score    Count      TP/TN/FP/FN
------------------------------------------------------------------------
                  acceleration     0.997      0.351     0.541     0.426        57 20/19894/17/37
                      activity     0.999      0.684     0.684     0.684        19 13/19943/6/6
                       battery     0.999      0.000     0.000     0.000         7 0/19951/10/7
                        button     1.000      0.500     0.875     0.636        14 7/19953/1/7
              colorTemperature     1.000      0.667     0.800     0.727         6 4/19961/1/2
                       contact     0.996      0.636     0.848     0.727       176 112/19772/20/64
                         level     0.999      0.333     0.500     0.400        27 9/19932/9/18
                          lock     0.996      0.771     0.287     0.419        35 27/19866/67/8
                        motion     0.981      0.000     0.000   

In [ ]:
# for i in [1] :
for i in range(len(lstm_tests)) :
    print( "==================HOME Case : %s =============" % test_names[i])
    lstm_pred= model2.predict( lstm_tests[i][0])
    
#     print_info( lstm_tests[i][1], lstm_pred, classes , confidance=0.7)
    print_info( lstm_tests[i][1], lstm_pred, classes , confidance=0.5)
    

In [ ]:
for i in [1] :
# for i in range(len(lstm_tests)) :
    print( "==================HOME Case : %s =============" % test_names[i])
    lstm_pred= model2.predict( lstm_tests[i][0])
    
#     print_info( lstm_tests[i][1], lstm_pred, classes , confidance=0.7)
    print_info( lstm_tests[i][1], lstm_pred, classes , confidance=0.992)
    



In [ ]:
lstm_pred =model2.predict( x_lstm_prossed_test2)


In [ ]:
print_info(y_lstm_prossed_test, lstm_pred, classes, confidance=0.60)

In [ ]:
lstm_pred =model2.predict( x_lstm_prossed_test)
print_info(y_lstm_prossed_test, lstm_pred, classes, confidance=0.6)

In [ ]:
lstm_pred = model2.predict( x_lstm_prossed_test)
print_info(y_lstm_prossed_test, lstm_pred, classes, confidance=0.6)

In [ ]:
print_info(y_lstm_prossed_train, y_lstm_prossed_train, classes, confidance=0.60)

In [ ]:
[x for x  in lstm_pred  if  np.sum(x) > 0 ]

In [ ]:
model2.save( "LSTM_withSigmoid_LargeData_F%s_E%d_B%d_M%s_%r" %
            (
            FoldID,
                Epoch_count,
                Batch_size,
                Mapper,
                IgnoreEmpty
            ) 
           )

In [ ]:
lstm_pred = model2.predict( x_lstm_prossed_test )

In [ ]:
print_info(y_lstm_prossed_test, lstm_pred, classes)

In [ ]:
# x_train_muhammed,y_train_muhammed, classes = pre_process_raw( x_train, y_train , dim_size, zero_pad=True, normalize=False)
# x_test_muhammed,y_test_muhammed, classes = pre_process_raw( x_test, y_test , dim_size, zero_pad=True, normalize=False)

In [ ]:
lstm_pred = model2.predict( x_lstm_prossed_test )
print_info(y_lstm_prossed_test, lstm_pred, classes)

In [ ]:
lstm_pred = model2.predict( x_lstm_prossed_test )
print_info(y_lstm_prossed_test, lstm_pred, classes)

In [ ]:
pred[:10]

In [ ]:
print_info(y_lstm_prossed_test, pred, classes)

In [ ]:
pred[0]

In [ ]:
len( y_lstm_prossed_train[0] )

In [ ]:
dim_size =160
x_lstm_prossed_train,y_lstm_prossed_train, _ = pre_process_raw( x_train, y_train , dim_size, zero_pad=True, normalize=False,classes=classes)
x_lstm_prossed_test,y_lstm_prossed_test, _ = pre_process_raw( x_test, y_test , dim_size, zero_pad=True, normalize=False,classes=classes)

In [ ]:
len([x  for x  in y_lstm_prossed_test if x[21]==1 or x[20]==1]), len(y_lstm_prossed_test  ) , len([x  for x  in y_lstm_prossed_test if x[21]==1 or x[20]==1])/len(y_lstm_prossed_test  ) *1.0

In [ ]:
[ x for x  in  pred if np.sum(x) > 1]

In [ ]:
len(y_lstm_prossed_test)

In [ ]:
def do_for_raun( pred   ):
    pp = pred
    pp[pp>=0.5] = 1
    pp[pp<0.5] = 0
    return pp

In [ ]:
len([x for x in pred if np.sum( do_for_raun(x) )==0 ])

In [ ]:
len([x for x in pred if  do_for_raun(x)[20] ==1 or do_for_raun(x)[21] ==1 ])

In [ ]:

# np.save("../files/muhammed/x_train.json" , x_train_muhammed)
# np.save("../files/muhammed/y_train.json", )


# np.save( "../files/muhammed/x_train.json", x_train_muhammed )
# np.save("../files/muhammed/y_train.json",  y_train_muhammed )
# np.save( "../files/muhammed/x_test.json",x_test_muhammed )
# np.save( "../files/muhammed/y_test.json",y_test_muhammed )
# np.save( "../files/muhammed/classes.json",  classes )


In [ ]:
len(x_lstm_prossed_test) + len(x_lstm_prossed_train)